In [ ]:
!pip install -i https://test.pypi.org/simple/ stock-screener-joncavme==0.0.1

# Stock-screener examples
Here are some basic examples on how to use stock-screener. The goal of the package is to provide some sort of simple library or framework for rapid development of your own scanners.
## Very basic use case:


In [ ]:
# A ready-made scanner class
from stock_screener.scanner import BasicScanner
"""
A ready-made condition that determines whether a stock
is above its 150- and 200-day simple moving average
"""
from stock_screener.condition.Above150And200SMA import Above150And200SMA

# Which universe's stocks will we look at? NASDAQ
universe = 'nasdaq'
"""
Stock-screener downloads these stocks' historical
data to a local directory. Let's download it to a
directory called "nasdaq" in the current directory.
"""
path = f'./{index}'

Let us instantiate the scanner. This particular scanner takes the universe and a list of Conditions as arguments:

In [ ]:
scanner = BasicScanner(index, [Above150And200SMA])

Then, we download the stock data.

In [ ]:
scanner.loadData(path)


Now we can get the stocks that conform to the given condition:

In [ ]:
candidates = scanner.getCandidates()

The method calls can also be chained to avoid redundant code:

In [ ]:
candidates = (
    BasicScanner(index, [Above150And200SMA])
    .loadData(path)
    .getCandidates()
)

## Creating your own Conditions
You can easily create your own conditions to use in the scanner:

In [ ]:
# Imported for type hinting
from stock_screener import Stock
from stock_screener.scanner import BasicScanner
# Abstract parent class our Condition will inherit
from stock_screener.condition import Condition

To create a condition, you need to write a child class of the imported Condition class, and implement the `__init__` and `fulfilled` method. The constructor must accept a Stock instance, and should generally create private attributes later used in the `fulfilled` method. The `fulfilled` method should solely be concerned with assessing the truthness of the condition being fulfilled.

In [ ]:
class Consolidating(Condition):
    def __init__(self, stock: Stock) -> None:
        """
        Always call super in the constructor.
        """
        super().__init__(stock)

        # We will look from the last close to 10 days back
        window = 10
        try:
            # Find the max and min closes in this window
            self.max_close = stock.getClose()[-window:].max()
            self.min_close = stock.getClose()[-window:].min()
        except IndexError:
            return False

    def fulfilled(self) -> bool:
        """
        If the difference between them is less than 3%
        we consider the stock consolidated
        """
        return self.min_close > (self.max_close * 0.97)

In [ ]:
print("Looking for consolidated stocks.")
candidates = (
    BasicScanner(index, [Consolidating])
    .loadData(path)
    .getCandidates()
)

print(list(map(lambda x: x.getTicker(), candidates)))